In [1]:
import pandas as pd
from itertools import combinations
from tqdm.auto import tqdm
islice = pd.IndexSlice

In [2]:
data = pd.read_json('../data/data-small.json')
q_to_score = data.set_index(['question', 'user'])
avgs = data.groupby('user').score.sum().rename('avg')

In [3]:
all_qs = list(data.question.unique())
K = 5
corrs = []
for qs in tqdm(list(combinations(all_qs, K))):
    qs_data = q_to_score.loc[islice[qs,:],:].swaplevel()
    answered_all = qs_data.groupby(level=[0]).size() == K
    answered_all = answered_all[answered_all].index
    qs_total = qs_data.loc[islice[answered_all,:]].groupby(level=[0]).sum().rename(columns={'score': 'qs'})
    r = qs_total.join(avgs).corr().qs.avg
    corrs.append({'qs': qs, 'r': r})

  0%|          | 0/15504 [00:00<?, ?it/s]